In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1J4M6fiuYXhaJKTqCBl9hWCqhD62773Dm/view?usp=sharing

In [ ]:
!unzip "BTK2024 - YOUNG BOYS.zip"

# Preprocessing the dataset

In [ ]:
!pip install -q catboost optuna unidecode

## Filepaths

In [ ]:
from pathlib import Path

DATASET_PATH = Path("datasets")

DUZELTILMIS_SEHIRLER_PATH = DATASET_PATH / "duzeltilmis_sehirler.json"
DRDATASTATS_PATH = DATASET_PATH / "drdatastats (1).xlsx"
TRAIN_PATH = DATASET_PATH / "train.csv"
TEST_PATH = DATASET_PATH / "test_x.csv"

DUZELTILMIS_UNIVERSITELER_PATH = DATASET_PATH / 'duzeltilmis_universiteler.json'
UNIVERSITE_ESLESMELELERI_PATH = DATASET_PATH /  'universite_eslesmeleri.json'
MESLEKLER_PATH = DATASET_PATH / 'meslekler.json'
LISE_SOZLUK_PATH = DATASET_PATH / 'lise_sozluk.json'
LISTE_BURS_PATH = DATASET_PATH / 'liste_burs.json'
JSON_FILE_LISE_SEHIR_PATH = DATASET_PATH / 'json_file_lise_sehir.json'

## Loading the modules

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity

from catboost import CatBoostRegressor, Pool
import optuna
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

from unidecode import unidecode
import re

## Loading the data

In [ ]:
import json

with open(DUZELTILMIS_SEHIRLER_PATH, "r") as f:
    duzeltilmis_sehirler = json.load(f)

with open(DUZELTILMIS_UNIVERSITELER_PATH, 'r') as f:
    duzeltilmis_universiteler = json.load(f)

with open(UNIVERSITE_ESLESMELELERI_PATH, 'r') as f:
    universite_eslesmeleri = json.load(f)

with open(MESLEKLER_PATH, 'r') as f:
    meslekler = json.load(f)

with open(LISE_SOZLUK_PATH, 'r') as f:
    lise_sözlük = json.load(f)

with open(LISTE_BURS_PATH, 'r') as f:
    liste_burs = json.load(f)

with open(JSON_FILE_LISE_SEHIR_PATH, 'r') as f:
    json_file_lise_sehir = json.load(f)

In [ ]:
# Veri yükleme
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
train = train.dropna(subset='Degerlendirme Puani').reset_index(drop=True)
df = pd.concat([train,test])
df.columns = df.columns.str.replace(r'[^a-zA-Z\s]', '', regex=True)

df.replace([None], np.nan, inplace=True)

## Feature engineering

yapabildigimiz her feature'da degerleri normalize edip, bilinmeyen degerleri  `bilinmiyor` diye isaretliyoruz.

#### DogumYeri

In [ ]:
# 'Dogum Yeri' sütununu işlemek için hazırlık yapıyoruz
col='Dogum Yeri'

# İl ve ilçe bilgilerini içeren Excel dosyasını okuyoruz
il_ilce_df = pd.read_excel(DRDATASTATS_PATH)

# İlk satırı (muhtemelen başlık satırı) atlıyoruz
il_ilce_df = il_ilce_df.iloc[1:]

# Sütun isimlerini 'il' ve 'ilçe' olarak belirliyoruz
il_ilce_df.columns=['il','ilçe']

# 'Dogum Yeri' sütunundaki boş değerleri 'bilinmiyor' ile dolduruyoruz
df[col] = df[col].fillna('bilinmiyor')

In [ ]:

# Metni normalize eden bir fonksiyon yazalım
def normalize_text(text):
    text = text.lower()  # Küçük harfe çevirme
    #text = to_ascii(text)
    text = unidecode(text)  # Türkçe karakterleri İngilizce karakterlere çevirme
    text = re.sub(r'[^a-z\s]', '', text)

    return text

In [ ]:


df[col] = df[col].apply(normalize_text)
# gelmisken bunlari da normalize edelim
il_ilce_df['il'] = il_ilce_df['il'].str.lower().apply(normalize_text)
il_ilce_df['ilçe'] = il_ilce_df['ilçe'].str.lower().apply(normalize_text)

In [ ]:
# ilçeleri illere eşleyen bir sözlük oluşturuyoruz
ilce_il_dict = pd.Series(il_ilce_df['il'].values, index=il_ilce_df['ilçe'].str.lower()).to_dict()

# 'dogum yeri' sütunundaki her değer için, eğer bir il adı içeriyorsa o ili, yoksa kendisini alıyoruz
df['dg2'] = df[col].apply(lambda x: next((yer for yer in il_ilce_df['il'].tolist() if yer in x), x))

# ilçe adlarını il adlarına dönüştürüyoruz, eğer ilçe değilse orijinal değeri koruyoruz
df['dg2'] = df['dg2'].map(ilce_il_dict).fillna(df['dg2'])

# tüm illerin listesini oluşturuyoruz
il_listesi = il_ilce_df['il'].unique()

# eğer 'dg2' sütunundaki değer bir il adı içermiyorsa, orijinal değeri koruyoruz
df['dg2'] = df['dg2'].apply(lambda x: x if any(il in x for il in il_listesi) else x)


In [ ]:
df[df[col]=='kdz eregli']['dg2'].map(duzeltilmis_sehirler)

In [ ]:
df['dg3'] = df['dg2'].map(duzeltilmis_sehirler)
df['dg3'] = df['dg3'].fillna(df['dg2'])

df['dg3'] = df['dg3'].apply(normalize_text)
df['dg3'] = df['dg3'].apply(lambda x: x if x in il_listesi else 'bilinmiyor')

In [ ]:
df['dg3'].nunique()

In [ ]:
df = df.drop(columns=['dg2'])

In [ ]:
df.to_csv('data1_dogumyeri.csv')

#### Ikametgah Sehri

In [ ]:
col='Ikametgah Sehri'

In [ ]:
df[col] = df[col].fillna('bilinmiyor')

# 'Blm' sütunundaki tüm değerleri normalize edelim
df[col] = df[col].apply(normalize_text)

# il ve ilçe isimlerini küçük harfe çevirip normalize et
il_ilce_df['il'] = il_ilce_df['il'].str.lower().apply(normalize_text)
il_ilce_df['ilçe'] = il_ilce_df['ilçe'].str.lower().apply(normalize_text)

# ilçe-il eşleştirme sözlüğü oluştur
ilce_il_dict = pd.Series(il_ilce_df['il'].values, index=il_ilce_df['ilçe'].str.lower()).to_dict()

# her bir değer için il bul, bulamazsa değeri olduğu gibi bırak
df['dg2'] = df[col].apply(lambda x: next((yer for yer in il_ilce_df['il'].tolist() if yer in x), x))

# ilçeleri illere dönüştür, bulamazsa orijinal değeri kullan
df['dg2'] = df['dg2'].map(ilce_il_dict).fillna(df['dg2'])

# unique il listesi oluştur
il_listesi = il_ilce_df['il'].unique()

# il listesinde olmayan değerleri 'bilinmiyor' olarak işaretle
df['dg2'] = df['dg2'].apply(lambda x: x if any(il in x for il in il_listesi) else 'bilinmiyor')

df = df.rename(columns={'dg3':'Dogum Yeri2',
                  'dg2':'Ikametgah Sehri2'})

df.to_csv('data2_Ikametgah_Sehri.csv',index=False)
df.head()

In [ ]:
df[col]

#### Lise Sehir

In [ ]:
# 'lise sehir' sütunu için işlemler yapılıyor
col='Lise Sehir'

# eksik değerleri 'bilinmiyor' ile dolduruyoruz
df[col] = df[col].fillna('bilinmiyor')

# 'bilinmiyor' değerine sahip satırların sayısını gösteriyoruz
print(df[df[col]=='bilinmiyor'].shape)

# metni normalize ediyoruz
df[col] = df[col].apply(normalize_text)

df[col].nunique()

In [ ]:
ilce_il_dict = pd.Series(il_ilce_df['il'].values, index=il_ilce_df['ilçe'].str.lower()).to_dict()
il_listesi = il_ilce_df['il'].unique()

df['dg2'] = df[col].apply(lambda x: next((yer for yer in il_ilce_df['il'].tolist() if yer in x), x))
df['dg2'] = df['dg2'].map(ilce_il_dict).fillna(df['dg2'])
df['dg2'] = df['dg2'].apply(lambda x: x if any(il in x for il in il_listesi) else x)
df['dg2'].nunique()

In [ ]:
df['dg3'] = df['dg2'].map(duzeltilmis_sehirler)
df['dg3'] = df['dg3'].fillna(df['dg2'])
df['dg3'] = df['dg3'].apply(normalize_text)
df['dg3'] = df['dg3'].apply(lambda x: x if x in il_listesi else 'bilinmiyor')

In [ ]:
# 'kahramanmaas' değerine sahip satırları seçip, belirli sütunları gösteriyoruz
# 'lise sehir' sütununda 'kahramanmaas' değeri olan satırları filtreleyip
# böylece kahramanmaraş için yapılan düzeltmeleri kontrol edebiliriz
df[df[col]=='kahramanmaas'][[col,'dg2','dg3']]

In [ ]:
df[df['dg2']=='bilinmiyor'][[col,'dg2','dg3']]

In [ ]:
df = df.rename(columns={'dg3':'Lise Sehir2'})
df = df.drop(columns=['dg2'])
df.to_csv('data3_Lise_Sehir.csv',index=False)

#### Universite adi 2

In [ ]:
# bu donusumler klasik artik

col='Universite Adi'
df[col+'2'] = df[col].fillna('bilinmiyor')
df[col+'2'] = df[col+'2'].apply(normalize_text)
df[col+'2'].nunique()

In [ ]:
df[col+'2'] = df[col+'2'].map(duzeltilmis_universiteler)
df[col+'2'].nunique()


In [ ]:
df[col+'2'] = df[col+'2'].map(universite_eslesmeleri).fillna(df[col+'2'])

In [ ]:
df[col+'2'].nunique()

In [ ]:
df.to_csv('data4_Universite_Adi2.csv',index=False)

#### Bolum

In [ ]:
col='Blm'
df[col+'2'] = df[col].fillna('bilinmiyor')
df[col+'2'] = df[col+'2'].apply(normalize_text)
df[col+'2'].unique()
# Kelimeleri ayırıp explode edelim
words = df[col+'2'].str.split().explode()

# Her kelimenin value_counts'u
word_counts = words.value_counts()

# Sonuçları görüntüleyelim
print(word_counts)


In [ ]:
# Eğer meslekler bir listeyse, her bir eleman için fonksiyonu uygula
meslekler_normalized = [normalize_text(meslek) for meslek in meslekler]

df1 = pd.DataFrame(word_counts).reset_index(names=['Blm2'])
liste_meslek = df1[df1[col+'2'].isin(meslekler_normalized)]['Blm2'].unique()
len(liste_meslek)

In [ ]:
for meslek in liste_meslek:
    df[meslek+'2'] = df[col+'2'].apply(lambda x: 1 if meslek in str(x).lower() else 0)



In [ ]:
df.shape

In [ ]:
df.to_csv('data5_Blm.csv',index=True)

In [ ]:
df.shape

In [ ]:
df[df['bankaci2']==1][[col+'2']]

In [ ]:
df1.head(500)['Blm2'].unique()

#### Lise Adi

In [ ]:
col='Lise Adi'
df[col].nunique()
df[col+'2'] = df[col].fillna('bilinmiyor')

df[col+'2'] = df[col+'2'].apply(normalize_text)


In [ ]:

# Bu bölümde, lise adlarını gruplamak için metin benzerliği ve kümeleme yöntemlerini kullandık.

# Adım 1: TF-IDF vektörlerine dönüştürme
# TF-IDF (Term Frequency-Inverse Document Frequency) yöntemi, metinleri sayısal vektörlere dönüştürür.
# Bu, kelimelerin önem derecesini belirler ve benzerlik hesaplamalarında kullanılır.
tfidf = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf.fit_transform(df[col+'2'].unique())

# Adım 2: Kosinüs benzerliği hesaplama
# Kosinüs benzerliği, vektörler arasındaki açıyı ölçer ve metin benzerliğini belirlemede kullanılır.
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# Adım 3: Aglomeratif Kümeleme ile gruplama
# Aglomeratif Kümeleme, hiyerarşik bir kümeleme yöntemidir ve benzer öğeleri gruplar.
# Bu yöntem, önceden küme sayısı belirtmeye gerek kalmadan, benzerlik eşiğine göre gruplar oluşturur.
# Ayrıca, farklı bağlantı kriterleri (örneğin, ortalama bağlantı) kullanarak esnek kümeleme sağlar.
clustering_model = AgglomerativeClustering(n_clusters=None, metric='precomputed', linkage='average', distance_threshold=0.8)
clusters = clustering_model.fit_predict(1 - cosine_sim_matrix)  # 1 - kosinüs benzerliği bize uzaklığı verir

# AgglomerativeClustering'in seçilme nedenleri:
# 1. Hiyerarşik yapı: Veri içindeki hiyerarşik ilişkileri yakalamak için uygundur.
# 2. Esneklik: Küme sayısını önceden belirtmeye gerek yoktur, benzerlik eşiği kullanılabilir.
# 3. Farklı bağlantı kriterleri: Ortalama, tam veya tek bağlantı gibi farklı stratejiler kullanılabilir.
# 4. Gürültüye karşı dayanıklılık: Aykırı değerlere karşı diğer yöntemlere göre daha az hassastır.
# 5. Yorumlanabilirlik: Oluşan kümelerin hiyerarşisi, veri yapısını anlamada yardımcı olur.


In [ ]:
df_lise = pd.DataFrame()
df_lise['unique'] = df[col+'2'].unique()
df_lise['clusters'] = clusters

result_dict = df_lise.set_index('unique')['clusters'].to_dict()
df[col+'2'] = df[col+'2'].map(result_dict)


df[col+'2'].isna().sum()

In [ ]:
df_lise[df_lise.clusters==356]['unique'].unique()

In [ ]:
df_lise.clusters.nunique()

In [ ]:
df.to_csv('data6_Lise_Adi.csv',index=False)

#### Lise Bolumu

In [ ]:
df = pd.read_csv('data6_Lise_Adi.csv')

col='Lise Bolumu'

df[col].nunique()
df[col+'2'] = df[col].fillna('bilinmiyor')

df[col+'2'] = df[col+'2'].apply(normalize_text)

df[col+'2'].unique()



In [ ]:
df[col+'2'] = df[col+'2'].map(lise_sözlük).fillna(df[col+'2'])

In [ ]:
df[col+'2'].nunique()

In [ ]:
df.to_csv('data7_Lise_Bolumu.csv',index=False)

#### Burs Aldigi Baska Kurum

In [ ]:
col='Burs Aldigi Baska Kurum'
df[col].nunique()

In [ ]:
df[col+'2'] = df[col].fillna('bilinmiyor')
df[col+'2'] = df[col+'2'].apply(normalize_text)

# Kelimeleri ayırıp explode edelim
words = df[col+'2'].str.split().explode()

# Her kelimenin value_counts'u
word_counts = words.value_counts()

# Sonuçları görüntüleyelim
word_counts.head(200)




In [ ]:
df['kyk'] = df[col+'2'].str.contains('kyk|kredi|yurt|ky|yk|kek|kvk|kre|kuk|kred|devlet|kredi yurtlar kurumu|kuru', case=False, na=False).astype(int)


In [ ]:
for word in liste_burs:
    df[word+'bs'] = df[col+'2'].str.contains(word, case=False, na=False).astype(int)

df['diyanetbs'].value_counts()


In [ ]:
df.to_csv('data8_burs_aldıgı_bk.csv',index=False)

#### Hangi STKnin Uyesisiniz

In [ ]:
col='Hangi STKnin Uyesisiniz'
df[col].nunique()

In [ ]:
df[col+'2'] = df[col].fillna('bilinmiyor')
df[col+'2'] = df[col+'2'].apply(normalize_text)
df[col+'2'].nunique()

In [ ]:
def cluster_mao(df,col,threshold):
    # Step 2: Convert text to TF-IDF vectors
    tfidf = TfidfVectorizer(max_features=1000)
    tfidf_matrix = tfidf.fit_transform(df[col+'2'].unique())

    # Step 3: Compute cosine similarity
    cosine_sim_matrix = cosine_similarity(tfidf_matrix)

    # Step 4: Perform clustering using Agglomerative Clustering
    clustering_model = AgglomerativeClustering(n_clusters=None, metric='precomputed', linkage='average', distance_threshold=threshold)
    clusters = clustering_model.fit_predict(1 - cosine_sim_matrix)  # 1 - cosine similarity gives us distance

    df_clustered = pd.DataFrame()
    df_clustered['unique'] = df[col+'2'].unique()
    df_clustered['clusters'] = clusters
    result_dict = df_clustered.set_index('unique')['clusters'].to_dict()
    df[col+'2'+str(threshold)[2:]] = df[col+'2'].map(result_dict)

cluster_mao(df,col,0.80)
cluster_mao(df,col,0.85)
cluster_mao(df,col,0.90)
cluster_mao(df,col,0.95)

In [ ]:
df.columns

In [ ]:

df['Hangi STKnin Uyesisiniz28'].nunique()

In [ ]:
df['Hangi STKnin Uyesisiniz285'].nunique()

In [ ]:
df.to_csv('data9_Hangi_STKnin_Uyesisiniz.csv',index=False)

#### Uye Oldugunuz Kulubun Ismi

In [ ]:
df = pd.read_csv('data9_Hangi_STKnin_Uyesisiniz.csv')

In [ ]:
col='Uye Oldugunuz Kulubun Ismi'
df[col].nunique()


In [ ]:
df[col+'2'] = df[col].fillna('bilinmiyor')
df[col+'2'] = df[col+'2'].apply(normalize_text)
df[col+'2'].nunique()



In [ ]:
cluster_mao(df,col,0.80)
cluster_mao(df,col,0.85)
cluster_mao(df,col,0.90)

In [ ]:
df.to_csv('data10_Uye_Oldugunuz_Kulubun_Ismi.csv',index=False)

#### Daha Onceden Mezun Olunduysa Mezun Olunan Universite ve Girisimcilikle Ilgili Deneyiminizi Aciklayabilir Misiniz

In [ ]:
df = pd.read_csv('data10_Uye_Oldugunuz_Kulubun_Ismi.csv')

In [ ]:
col='Daha nceden Mezun Olunduysa Mezun Olunan niversite'
df[col + '2'] = df[col].apply(lambda x: 0 if pd.isnull(x) else 1)
df[col].nunique()

In [ ]:
col='Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz'
df[col].nunique()

In [ ]:
df[col+'2'] = df[col].fillna('bilinmiyor')
df[col+'2'] = df[col+'2'].apply(normalize_text)
df[col+'2'].unique()

In [ ]:
df[col+'2'+'count'] = df[col+'2'].apply(lambda x: len(x.split()))

In [ ]:
for i in range(5, 11):
    col_name = col+'2' + 'count' + str(i)
    df[col_name] = df[col+'2'].apply(lambda x: sum(1 for kelime in x.split() if len(kelime) > i))


In [ ]:
df.to_csv('data11_Girisimcilikle_Ilgili_Deneyiminizi.csv',index=False)

#### Dogum Tarihi

In [ ]:
col='Dogum Tarihi'

In [ ]:
# df['Dogum Tarihi'] = df['Dogum Tarihi'].astype(str)
# df['Dogum Tarihi'] = df['Dogum Tarihi'].fillna()
import re
def extract_year_combined(date_str):
    if not isinstance(date_str, str):
        return np.nan

    # Remove extra spaces
    date_str = date_str.strip()

    # Try to find a year in the format YYYY
    year_match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if year_match:
        return int(year_match.group(0))

    # Handle special cases where month names might be present (e.g., "Kasım" or "November")
    month_names = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık',
                   'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

    # If the date contains a Turkish or English month name, try to extract the year
    if any(month in date_str for month in month_names):
        year_match = re.search(r'\b(19|20)\d{2}\b', date_str)
        if year_match:
            return int(year_match.group(0))

    # Handle known formats: 'dd-MMM-yy' or 'MM/DD/YY HH:MM'
    try:
        # First try 'dd-MMM-yy' format
        parsed_date = pd.to_datetime(date_str, format='%d-%b-%y', errors='coerce')
        if not pd.isna(parsed_date):
            return parsed_date.year

        # Try 'MM/DD/YY HH:MM' format
        parsed_date = pd.to_datetime(date_str, format='%m/%d/%y %H:%M', errors='coerce')
        if not pd.isna(parsed_date):
            return parsed_date.year
    except:
        pass

    # If no valid year is found, return NaN
    return np.nan

# Apply the improved function to the 'unique_dates' column
df['dogum_clean'] = df['Dogum Tarihi'].apply(extract_year_combined)
#df['dogum_clean'] = df['dogum_clean'].astype(int)
df.loc[df['dogum_clean']<0,'dogum_clean']=np.nan
df.loc[df['dogum_clean']<1990,'dogum_clean']=np.nan
df.loc[df['dogum_clean']>2005,'dogum_clean']=np.nan

In [ ]:
df.to_csv('data12_Dogum_Tarihi.csv',index=False)

## Modelleme oncesi verilerin encodelanmasi

#### Cinsiyet

In [ ]:
col='Cinsiyet'
df[col] = df[col].str.lower()

df[col] = df[col].fillna('belirtmek istemiyorum')
df[col].unique()

df[col] = df[col].map({
    'erkek': 1,
    'kadın': 2,
    'belirtmek istemiyorum': 0,
})
df[col].unique()

#### Burslu ise Burs Yuzdesi

In [ ]:
col='Burslu ise Burs Yuzdesi'
#df[col] = df[col].str.lower()

df[col] = df[col].fillna(0)
df[col].unique()

#### Universite Kacinci Sinif

In [ ]:
col='Universite Kacinci Sinif'
df[col] = df[col].str.lower()

df[col] = df[col].fillna('-1')
df[col].unique()

mapping = {
    '-1':0,
    'hazırlık': 0,
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    'mezun': 7,
    'tez': 8,
    'yüksek lisans': 9
}

df[col] = df[col].map(mapping)
df[col].unique()

#### Stk Projesine Katildiniz Mi

In [ ]:
col='Stk Projesine Katildiniz Mi'
print(df[col].unique())
df[col] = df[col].map({'Evet': 1, 'Hayır': 0}).fillna(0)
print(df[col].unique())

#### Ingilizce Seviyeniz

In [ ]:
col='Ingilizce Seviyeniz'
print(df[col].unique())
mapping = {'Başlangıç': 1, '0': 2, 'Orta': 3, 'İleri': 4}
df[col] = df[col].map(mapping).fillna(-1)
print(df[col].unique())


#### Ingilizce Biliyor musunuz

In [ ]:
col='Ingilizce Biliyor musunuz'
print(df[col].unique())

mapping = {'Evet': 1, 'Hayır': 0}
df[col] = df[col].map(mapping).fillna(-1)
print(df[col].unique())


#### Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz

In [ ]:
col='Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz'
df[col].unique()
mapping = {'Evet': 1, 'Hayır': 0}
df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Universite Not Ortalamasi

In [ ]:
col='Universite Not Ortalamasi'
df[col].unique()

mapping = {
    '4.0-3.5': 3.75, '4-3.5': 3.75,
    '3.50-3': 3.25, '3.50 - 4.00': 3.25,
    '3.00-2.50': 3.00, '3.00 - 3.50': 3.00, '3.00 - 4.00': 3.00, '3.00 - 3.49': 3.00,
    '2.50 ve altı': 2.75, '2.50 -3.00': 2.75, '2.50 - 3.00': 2.75, '2.50 - 2.99': 2.75,
    '2.00 - 2.50': 2.25, '1.80 - 2.49': 2.25,
    '1.00 - 2.50': 1.75,
    '0 - 1.79': 1.00,
    'Ortalama bulunmuyor': -1, 'ORTALAMA BULUNMUYOR': -1, 'Not ortalaması yok': -1, 'Hazırlığım': -1
}

df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Girisimcilikle Ilgili Deneyiminiz Var Mi

In [ ]:
col='Girisimcilikle Ilgili Deneyiminiz Var Mi'
df[col].unique()

mapping = {'Evet': 1, 'Hayır': 0}
df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Lise Mezuniyet Notu

In [ ]:
col='Lise Mezuniyet Notu'
df[col].unique()

mapping = {
    '3.50-3': 3.25, '3.00 - 4.00': 3.50, '3.00-2.50': 2.75, '2.50 ve altı': 2.50,
    '4.00-3.50': 3.75, '3.50-3.00': 3.25,
    '69-55': 62.0, '100-85': 92.5, '84-70': 77.0, '54-45': 49.5, '44-0': 22.0,
    '75 - 100': 87.5, '50 - 75': 62.5,
    'Not ortalaması yok': -1,
    '25 - 50': 37.5, '0 - 25': 12.5,
    '50 - 74': 62.0, '25 - 49': 37.0, '0 - 24': 12.0
}

df[col] = df[col].map(mapping).fillna(-1)


#### Universite Turu

In [ ]:
col='Universite Turu'
df[col] = df[col].str.lower()
df[col].unique()

mapping = {'özel': 1, 'devlet': 0}
df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Lise Turu

In [ ]:
col='Lise Turu'
df[col] = df[col].str.lower()
df[col].unique()

mapping = {
    'fen lisesi': 5,
    'anadolu lisesi': 4,
    'özel lisesi': 3, 'özel lise': 3, 'özel': 3,
    'düz lise': 2,
    'meslek lisesi': 2, 'meslek': 2,
    'i̇mam hatip lisesi': 1,
    'devlet': 1,
    'diğer': 0
}

df[col] = df[col].map(mapping).fillna(0)
df[col].unique()


#### Baska Bir Kurumdan Burs Aliyor mu

In [ ]:
col='Baska Bir Kurumdan Burs Aliyor mu'
df[col].unique()

mapping = {'Evet': 1, 'Hayır': 0}
df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Aktif olarak bir STK yesi misiniz

In [ ]:
col='Aktif olarak bir STK yesi misiniz'
df[col].unique()

mapping = {'Evet': 1, 'Hayır': 0}
df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Profesyonel Bir Spor Daliyla Mesgul musunuz

In [ ]:
col='Profesyonel Bir Spor Daliyla Mesgul musunuz'
df[col].unique()
mapping = {'Evet': 1, 'Hayır': 0}
df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Burs Aliyor mu

In [ ]:
col='Burs Aliyor mu'
df[col] = df[col].str.lower()
df[col].unique()
mapping = {'evet': 1, 'hayır': 0}
df[col] = df[col].map(mapping).fillna(-1)
df[col].unique()

#### Baska Kurumdan Aldigi Burs Miktari

In [ ]:
col='Baska Kurumdan Aldigi Burs Miktari'
#df[col] = df[col].str.lower()
df[col].unique()

mapping = {
    '300': 300,
    '300 TL': 300,
    '150 TL': 150,
    '430tlyılda 10 ay': 430,
    '400': 400,
    '250 TL': 250,
    'Toplamda 1000 lira fakat mezun oldugun icin devam etmiyor': 0,
    '850 TL': 850,
    '500 TL': 500,
    '300 tl': 300,
    'AYLIK 280 TL': 280,
    '430': 430,
    '1000 TL': 1000,
    '900 TL': 900,
    '300 LİRA': 300,
    '365': 365,
    '500TL, 300TL': 800,  # Ortalaması alındı
    '125 TL': 125,
    '850': 850,
    '180': 180,
    '260': 260,
    '300 lira': 300,
    '425 TL': 425,
    'aylık 100 tl': 100,
    '300 TL nakit + yemek + ulaşım': 300,
    '300TL': 300,
    '250': 250,
    '900': 900,
    '150': 150,
    '400 TL': 400,
    '800': 800,
    '300 tl aylık': 300,
    '280': 280,
    '400,00 TL': 400,
    '200 TL': 200,
    '430 tl': 430,
    '500 TL - 300 TL': 800,  # Ortalaması alındı
    '400 tl': 400,
    '325 TL': 325,
    '300 TL / Aylık': 300,
    '275 TLAYLIK': 275,
    '10 ay süre ile aylık 600 TL': 600,
    '500 tl ve 300 tl': 800,  # Ortalaması alındı
    '500 tl': 500,
    'BİN TÜRK LİRASI': 1000,
    '475': 475,
    '300,0 tl': 300,
    '675 TL': 675,
    'kredi-300': 800,
    'Aylık 300 TL': 300,
    '300-400': 350,  # Ortalaması alındı
    '750': 750,
    '330 TL': 330,
    'aylık 330 tl': 330,
    '490': 490,
    '320 TL': 320,
    '560 TL': 560,
    'Tam Burslu olduğum için yalnız eğitim ücreti ek burs almıyorum': 0,  # Burs yok
    '500': 500,
    '600 TL/Yurt Bursu/Kitap Bursu': 600,
    '430TL': 430,
    '350': 350,
    '600': 600,
    '8 Ay Boyunca Aylık 720 TL': 720,
    '330': 330,
    '250 tl': 250,
    '350 TL - Türkiye Girişimcilik Vakfından burs kazandığım takdirde Çelik İhracatçıları Birliğindeki bursumdan vazgeçeceğim.': 0,
    '150 TL/Ay': 150,
    '100 tl': 100,
    '300TL nakit + yemek ve yurt bursu': 1000,
    '10 ay süresince 600 TL cep harçlığı': 600,
    '590': 590,
    'Aylık 490 TL - 9 ay boyunca': 490,
    '1990': 1990,
    '475 TL': 475,
    '560?': 560,
    '600 TL': 600,
    '180 TL': 180,
    '560 tl': 560,
    '200': 200,
    'Nakit burslar 700TL, Nakde çevrilen yemek ve yol burslarıyla 1160': 1160,
    '330 tl': 330,
    'Üstün başarı bursu - 750 TL': 1750,
    '250 Tl': 250,
    '100 TL': 100,
    '535': 535,
    'aylık 200 tl': 200,
    '200 TL/ay': 200,
    '700 TL': 700,
    '1500': 1500,
    '600 okul,200 LCW': 800,  # LCW belirtilmediği için hariç tutuldu
    '2000': 2000,
    '620': 620,
    '175': 175,
    '325': 325,
    '570': 570,
    '1200': 1200,
    '1750': 1750,
    '610': 610,
    '275': 275,
    '520': 520,
    '460': 460,
    '515': 515,
    '1100': 1100,
    '820': 820,
    '815': 815,
    '450': 450,
    '240': 240,
    '470': 470,
    '60': 60,
    '670': 670,
    '315': 315,
    '480': 480,
    '650': 650,
    '1000': 1000,
    '780': 780,
    '920': 920,
    '300 ₺': 300,
    '425': 425,
    '440': 440,
    '670 Türk Lirası': 670,
    '2000 TL': 2000,
    '1275 TL': 1275,
    'Toplamda 1000': 1000,
    '750 TL': 750,
    '340': 340,
    '425TL': 425,
    '660 TL': 660,
    '425 tl': 425,
    '780, 425': 1225,  # Ortalaması alındı
    '380-425': 805,  # Ortalaması alındı
    '130': 130,
    '1120TRY (aylık)': 1120,
    'Nakit Destek Bursu (9 Ay süre ile 660 TL)': 660,
    '650 TL': 650,
    '670 tl': 670,
    '420': 420,
    '375': 375,
    '1275': 1275,
    'Yıllık belirlenen kapsamlı burs kadar': 1000,  # Belirsiz
    '350tl': 350,
    'Egitim aylari , her ay 250 TL': 250,
    '514': 514,
    '225': 225,
    '100% okul ve 50% yurt bursu': 2000,  # Yüzdeyle ifade edilen burslar
    '420,geri ödemeli': 420,
    '350 tl': 350,
    '175 TL': 175,
    '650TL + 400 TL': 1050,  # Ortalaması alındı
    '400, 425': 825,  # Ortalaması alındı
    '1075 toplam': 1075,
    '9*100': 900,
    '400TL': 400,
    '1000.00 TL': 1000,
    '100': 100,
    '220': 220,
    'Aylık 400 TL': 400,
    '420 Turk Lirasi': 420,
    '150 tl': 150,
    '425 Tl': 425,
    '2016-2017 yılı içinde aylık 400 Türk lirası': 400,
    '400 tl / ay': 400,
    '424': 424,
    '425 aylık': 425,
    '435': 435,
    'Aylık 500 TL': 500,
    '225 tl': 225
}
df[col] = df[col].map(mapping)
df[col]=df[col].fillna(0)
df[col].unique()

#### Kardes Sayisi

In [ ]:
col='Kardes Sayisi'
df[col].unique()

mapping = {
    '1': 1,
    '6': 6,
    '10': 10,
    '3': 3,
    '4': 4,
    '2': 2,
    '5': 5,
    '0': 0,
    '7': 7,
    '8': 8,
    '11': 11,
    '9': 9,
    '12': 12,
    '13': 13,
    'Kardeş Sayısı 1 Ek Bilgi Aile Hk. Anne Vefat': 1,  # Belirsiz bilgi
    '18': 18,
    '14': 14,
    4.0: 4,
    5.0: 5,
    1.0: 1,
    2.0: 2,
    6.0: 6,
    3.0: 3,
    9.0: 9,
    0.0: 0,
    10.0: 10,
    7.0: 7,
    8.0: 8,
    np.nan: 0
}
df[col] = df[col].map(mapping)
df[col].unique()

#### Spor Dalindaki Rolunuz Nedir

In [ ]:
col='Spor Dalindaki Rolunuz Nedir'
df[col].unique()

mapping = {
    'Diğer': 0,
    '0': 0,
    np.nan: 0,
    'Kaptan': 1,
    'KAPTAN / LİDER': 1,
    'DİĞER': 0,
    'Lider/Kaptan': 1,
    'Bireysel': 2,
    'Takım Oyuncusu': 3,
    '-': 0,
    'Bireysel Spor': 2
}
df[col] = df[col].map(mapping)
df[col].unique()

#### Baba Sektor

In [ ]:
col='Baba Sektor'
df[col] = df[col].str.lower()
df[col].unique()

mapping = {
    '0': 0,
    'özel sektör': 1,
    'kamu': 2,
    np.nan: 0,
    'diğer': 0,
    'di̇ğer': 0,
    '-': 0
}
df[col] = df[col].map(mapping)
df[col].unique()

#### Baba Calisma Durumu

In [ ]:
col='Baba Calisma Durumu'
df[col].unique()
mapping = {
    'Emekli': 0,   # Emekli için 2 atandı
    'Evet': 1,     # Evet için 1 atandı
    'Hayır': 0,    # Hayır için 0 atandı
    np.nan: 0     # NaN için -1 atandı
}

df[col] = df[col].map(mapping)
df[col].unique()

#### Baba Egitim Durumu

In [ ]:
col='Baba Egitim Durumu'
df[col] = df[col].str.lower()
df[col].unique()
mapping = {
    'eğitim yok': 0,
    'eği̇ti̇m yok': 0,
    'eğitimi yok': 0,
    '0': 0,
    'ilkokul mezunu': 1,
    'i̇lkokul': 1,
    'ortaokul mezunu': 2,
    'ortaokul': 2,
    'lise': 3,
    'lise mezunu': 3,
    'li̇se': 3,
    'üniversite': 4,
    'üniversite mezunu': 4,
    'üni̇versi̇te': 4,
    'yüksek lisans': 5,
    'yüksek lisans / doktora': 5,
    'yüksek lisans / doktara': 5,
    'yüksek li̇sans': 5,
    'doktora': 6,
    np.nan: 0
}
df[col] = df[col].map(mapping)
df[col].unique()
df[col] = df[col].fillna(0)
df[col].unique()

#### Anne Egitim Durumu

In [ ]:
col='Anne Egitim Durumu'
df[col] = df[col].str.lower()
df[col].unique()
mapping = {
    'eğitim yok': 0,
    'eği̇ti̇m yok': 0,
    'eğitimi yok': 0,
    'ilkokul mezunu': 1,
    'i̇lkokul': 1,
    'ortaokul mezunu': 2,
    'ortaokul': 2,
    'lise': 3,
    'lise mezunu': 3,
    'li̇se': 3,
    'üniversite': 4,
    'üniversite mezunu': 4,
    'üni̇versi̇te': 4,
    'yüksek lisans': 5,
    'yüksek lisans / doktora': 5,
    'yüksek lisans / doktara': 5,
    'yüksek li̇sans': 5,
    'doktora': 6,
    np.nan: 0
}

df[col] = df[col].map(mapping)
df[col].unique()
df[col] = df[col].fillna(0)
df[col].unique()

#### Anne Calisma Durumu

In [ ]:
col='Anne Calisma Durumu'
df[col].unique()
mapping = {
    'Hayır': 0,
    'Evet': 1,
    'Emekli': 0,
    np.nan: 0
}
df[col] = df[col].map(mapping)
df[col].unique()

#### Anne Sektor

In [ ]:
col='Anne Sektor'
df[col] = df[col].str.lower()
df[col].unique()
mapping = {
    '0': 0,
    'kamu': 1,
    'özel sektör': 2,
    'diğer': 0,
    'di̇ğer': 0,
    '-': 0,
    np.nan: 0
}
df[col] = df[col].map(mapping)
df[col].unique()

#### Daha Once Baska Bir Universiteden Mezun Olmus

In [ ]:
col='Daha Once Baska Bir Universiteden Mezun Olmus'
df[col].unique()
mapping = {
    'Hayır': 0,
    'Evet': 1,
    np.nan: 0
}
df[col] = df[col].map(mapping)
df[col].unique()

#### Lise Sehir

In [ ]:
df['Lise Sehir'] = df['Lise Sehir'].map(json_file_lise_sehir)

# Sonunda modellemeye gecebiliriz

In [ ]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna('bilinmiyor')
    df[col] = df[col].apply(normalize_text)

In [ ]:
df = df.iloc[:,1:]


In [ ]:
train = df[df['Degerlendirme Puani'].notna()]
test = df[df['Degerlendirme Puani'].isna()]

train.shape, test.shape

In [ ]:
# Hedef değişkeni ayırma
y = train['Degerlendirme Puani']
X = train.drop(['Degerlendirme Puani', 'id'], axis=1)

# Hedef değişkendeki NaN değerleri kontrol etme ve ele alma
print(f"NaN values in target variable before: {y.isnull().sum()}")
y = y.fillna(y.mean())  # NaN değerleri ortalama ile doldurma
print(f"NaN values in target variable after: {y.isnull().sum()}")

# Tarih sütunlarını işleme
date_columns = ['Dogum Tarihi']
for col in date_columns:
    X[col] = pd.to_datetime(X[col], errors='coerce')
    X[f'{col}_year'] = X[col].dt.year
    X[f'{col}_month'] = X[col].dt.month
    X[f'{col}_day'] = X[col].dt.day
    X.drop(col, axis=1, inplace=True)

    test[col] = pd.to_datetime(test[col], errors='coerce')
    test[f'{col}_year'] = test[col].dt.year
    test[f'{col}_month'] = test[col].dt.month
    test[f'{col}_day'] = test[col].dt.day
    test.drop(col, axis=1, inplace=True)

# Kategorik ve nümerik kolonları belirleme
cat_features = X.select_dtypes(include=['object']).columns
num_features = X.select_dtypes(include=['int64', 'float64']).columns

# Kategorik değişkenleri kodlama
for col in cat_features:
    # Eğitim ve test setlerindeki benzersiz değerleri birleştir
    unique_values = set(X[col].unique()) | set(test[col].unique())
    # Her iki sette de olmayan bir değer ekle (bilinmeyen kategoriler için)
    unique_values.add('Unknown')

    # LabelEncoder'ı bu genişletilmiş değer kümesiyle oluştur
    le = LabelEncoder()
    le.fit(list(unique_values))

    # Dönüşümü uygula, bilinmeyen değerleri 'Unknown' olarak işaretle
    X[col] = X[col].map(lambda x: x if x in le.classes_ else 'Unknown')
    X[col] = le.transform(X[col])

    test[col] = test[col].map(lambda x: x if x in le.classes_ else 'Unknown')
    test[col] = le.transform(test[col])

# Nümerik değişkenlerdeki NaN değerleri ele alma
for col in num_features:
    X[col] = X[col].fillna(X[col].mean())
    test[col] = test[col].fillna(X[col].mean())  # Test setini eğitim setinin ortalamasıyla doldur

In [ ]:

# # Veriyi eğitim ve doğrulama setlerine ayırma
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Optuna için objective fonksiyonu
# def objective(trial):
#     param = {
#         'iterations': trial.suggest_int('iterations', 1000, 1500),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
#         'depth': trial.suggest_int('depth', 5, 10),
#         #'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 100.0),
#         #'border_count': trial.suggest_int('border_count', 32, 255),
#         #'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 100.0),
#         #'random_strength': trial.suggest_loguniform('random_strength', 1e-8, 10.0),
#     }

#     model = CatBoostRegressor(**param, cat_features=list(cat_features), random_state=42)
#     model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=0)

#     return model.best_score_['validation']['RMSE']

# # Optuna çalıştırma
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=20) #1 ile çalıştırıldı

# # En iyi hiperparametrelerle model eğitimi
# best_params = study.best_params
# best_model = CatBoostRegressor(**best_params, cat_features=list(cat_features), random_state=42)
# best_model.fit(X, y, verbose=0)

# # # Çapraz doğrulama skorları
# # cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='neg_mean_squared_error')
# # cv_rmse_scores = np.sqrt(-cv_scores)

# # print("Çapraz Doğrulama RMSE Skorları:", cv_rmse_scores)
# # print("Ortalama Çapraz Doğrulama RMSE:", cv_rmse_scores.mean())
# # print("Çapraz Doğrulama RMSE Standart Sapması:", cv_rmse_scores.std())

# # Eğitim seti üzerinde performans
# train_predictions = best_model.predict(X)
# train_rmse = np.sqrt(mean_squared_error(y, train_predictions))
# train_r2 = r2_score(y, train_predictions)

# print("\nEğitim Seti Performansı:")
# print("RMSE:", train_rmse)
# print("R2 Score:", train_r2)

# # Test seti üzerinde tahmin yapma
# test_predictions = best_model.predict(test.drop('id', axis=1))

# # Sonuçları kaydetme
# submission = pd.DataFrame({'id': test['id'], 'Degerlendirme Puani': test_predictions})
# submission.to_csv('submission1.csv', index=False)

# print("\nTest Seti Tahminleri İstatistikleri:")
# print("Minimum:", test_predictions.min())
# print("Maksimum:", test_predictions.max())
# print("Ortalama:", test_predictions.mean())
# print("Medyan:", np.median(test_predictions))
# print("Standart Sapma:", test_predictions.std())

# print("\nModel eğitimi ve tahmin tamamlandı. Sonuçlar 'submission.csv' dosyasına kaydedildi.")

In [ ]:
random_state=[42,0,12,123,1, 7, 999, 2020, 2021, 101, 1234,
                 11, 13, 27, 100, 2022, 256, 512, 777, 888, 321]

for i in random_state:
    best_params =  {'iterations': 2000, 'learning_rate': 0.06631033720612822, 'depth': 7}
    best_model = CatBoostRegressor(**best_params, cat_features=list(cat_features), random_state=i)
    best_model.fit(X, y,verbose=1000)

    train_predictions = best_model.predict(X)
    train_rmse = np.sqrt(mean_squared_error(y, train_predictions))
    train_r2 = r2_score(y, train_predictions)

    print("\nEğitim Seti Performansı:")
    print("RMSE:", train_rmse)
    print("R2 Score:", train_r2)

    test = test[X.columns]
    test_predictions = best_model.predict(test)
    submission = pd.DataFrame({'id': pd.read_csv(DATASET_PATH / "test_x.csv")['id'], 'Degerlendirme Puani': test_predictions})
    submission.loc[submission['Degerlendirme Puani']<0,'Degerlendirme Puani']=0
    submission.to_csv(f'submission_show{i}.csv', index=False)

In [ ]:
import os
import pandas as pd

# Dosyaların bulunduğu dizin
directory = "./"
# submission_show ile başlayan CSV dosyalarını okuma
csv_files = [f for f in os.listdir(directory) if f.startswith('submission_show') and f.endswith('.csv')]

# Tüm CSV dosyalarını birleştirmek için bir liste oluşturun
csv_files

In [ ]:
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    dfs.append(df.iloc[:,1:])

# Tüm CSV dosyalarını birleştir
combined_df = pd.concat(dfs, ignore_index=True,axis=1)


In [ ]:
combined_df.iloc[:,:3].mean(axis=1).describe()

In [ ]:
combined_df.iloc[:,:5].mean(axis=1).describe()

In [ ]:
combined_df.iloc[:,:10].mean(axis=1).describe()

In [ ]:
import matplotlib.pyplot as plt

# Tek grafik içerisinde tüm sütunlar için histogram çizdir
plt.figure(figsize=(8, 5))

# Her sütunu aynı grafikte farklı renkte çizdir
for column in combined_df.select_dtypes(include='number').columns:
    plt.hist(combined_df[column].dropna(), bins=50,  label=column)

# Grafiği etiketle ve göster
plt.legend(loc='upper right')
plt.title('Histogram of All Columns')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


In [ ]:
submission = pd.DataFrame({'id': pd.read_csv(DATASET_PATH / "test_x.csv")['id'], 'Degerlendirme Puani': combined_df.mean(axis=1)})
submission.to_csv('submission_rs21.csv', index=False)
